# Blue Brain BioExplorer

![](../../bioexplorer_banner.png)

## Visualization of an RNA sequence (Work in progress!)

In [ ]:
from bioexplorer import BioExplorer, RNASequence, AnimationParams, Vector2, Vector3, Quaternion
import math
import os

be = BioExplorer()
be.reset_scene()
print(be.version())

In [ ]:
# Resources
resource_folder = os.path.abspath('../../../tests/test_files')
rna_folder = os.path.join(resource_folder, 'rna')
pdb_folder = os.path.join(resource_folder, 'pdb')

assembly_name = 'RNA assembly'
rna_name = assembly_name + '_' + be.NAME_RNA_SEQUENCE

In [ ]:
def add_rna_sequence(assembly_name, frame, rotation=Quaternion()):
    offset = 0.01 * math.cos(frame * math.pi / 180.0)

    rna_sequence = RNASequence(
        source=os.path.join(rna_folder, 'sars-cov-2.rna'),
        protein_source=os.path.join(pdb_folder, '7bv1.pdb'),
        shape=be.RNA_SHAPE_TREFOIL_KNOT,
        shape_params=Vector2(45.0, 1.0),
        curve_params=Vector3(1.51 + offset, 1.12 - offset, 1.93 + offset),
        animation_params=AnimationParams(frame, frame + 1, 0.1, frame + 2, 0.001),
        values_range=Vector2(-8.0 * math.pi, 8.0 * math.pi)
    )

    be.remove_assembly(name=assembly_name)
    be.add_assembly(
        name=assembly_name, rotation=rotation, clipping_planes=[[1.0, 0.0, 0.0, 10.0]])
        
    status = be.add_rna_sequence(
        assembly_name=assembly_name, 
        name=rna_name,
        rna_sequence=rna_sequence)

    return status

## Rendering settings

In [ ]:
status = be.set_rendering_quality(be.RENDERING_QUALITY_HIGH)

## Animation

In [ ]:
from bioexplorer import MovieMaker
mm = MovieMaker(be)
output_folder = '/tmp'

In [ ]:
mm.set_camera(
    direction=[0.0, 0.0, -1.0],
    origin=[-1.4913494291017158, -2.7600310873311957, 201.11775054410705],
    up=[0.0, 1.0, 0.0]
)

In [ ]:
from tqdm import tqdm

q_start = Quaternion(0.0, 1.0, 0.0, 0.0)
q_end = Quaternion(1.0, 0.0, 0.0, 0.0)

for frame in tqdm(range(0, 36)):
    add_rna_sequence(
        assembly_name=assembly_name, frame=frame,
        rotation=Quaternion.slerp(q_start, q_end, 
                                  0.1 * (0.5 + 0.5 * math.cos(float(frame) * math.pi / 18.0)))
    )

    '''Snapshot'''
    mm.create_snapshot(
        renderer='bio_explorer',
        size=[512, 512], samples_per_pixel=32,
        path=output_folder, base_name='%05d' % frame)